In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mpl
import seaborn as sea
import warnings
import datetime as dt

In [44]:
dataFrame = pd.read_csv("marketing_campaign.csv", sep = ";")

In [45]:
dataFrame.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,3,11,0


In [46]:
dataFrame.shape

(2240, 29)

In [47]:
pd.set_option("display.max_columns", None)

In [48]:
warnings.filterwarnings("ignore")

In [49]:
dataFrame = dataFrame.drop(["ID"], axis = 1)

In [50]:
dataFrame.shape

(2240, 28)

In [51]:
y = dataFrame[["Response"]]
x = dataFrame.drop(["Response"], axis = 1)

In [52]:
y.shape

(2240, 1)

In [53]:
x.shape

(2240, 27)

In [54]:
y.mean()

Response    0.149107
dtype: float64

In [55]:
reference_date = dt.datetime(2024, 5, 1)
print(reference_date)

2024-05-01 00:00:00


In [59]:
#convert year of birth to age, convert dt customer to time span of user being registered customer
x["Year_Birth"] = pd.to_datetime(dataFrame["Year_Birth"], format="%Y")
x["Age"] = round((reference_date - x["Year_Birth"]).dt.days / 365, 0)
x["Dt_Customer"] = pd.to_datetime(x["Dt_Customer"])
x["Customer_Tenure"] = round((reference_date - x["Dt_Customer"]).dt.days / 365, 0)

#x = x.drop(["Year_Birth", "Dt_Customer"], axis = 1)

In [60]:
x[["Age", "Customer_Tenure"]].head()

,Age,Customer_Tenure
0,67.0,12.0
1,70.0,10.0
2,59.0,11.0
3,40.0,10.0
4,43.0,10.0


In [61]:
x = x.drop(["Year_Birth", "Dt_Customer"], axis = 1)

In [62]:
dataFrame.dtypes

Year_Birth               int64
Education               object
Marital_Status          object
Income                 float64
Kidhome                  int64
Teenhome                 int64
Dt_Customer             object
Recency                  int64
MntWines                 int64
MntFruits                int64
MntMeatProducts          int64
MntFishProducts          int64
MntSweetProducts         int64
MntGoldProds             int64
NumDealsPurchases        int64
NumWebPurchases          int64
NumCatalogPurchases      int64
NumStorePurchases        int64
NumWebVisitsMonth        int64
AcceptedCmp3             int64
AcceptedCmp4             int64
AcceptedCmp5             int64
AcceptedCmp1             int64
AcceptedCmp2             int64
Complain                 int64
Z_CostContact            int64
Z_Revenue                int64
Response                 int64
dtype: object

In [63]:
#split features to categorical and numerical
numerical = x.select_dtypes(include = "number")
categorical = x.select_dtypes(include = "object")

In [65]:
def uniqueness(x):
    x = x.value_counts().count()
    return x
df_value_counts = pd.DataFrame(numerical.apply(lambda x: uniqueness(x)))

In [66]:
df_value_counts.columns = ["feature_levels"]

In [67]:
df_value_counts.head()

,feature_levels
Income,1974
Kidhome,3
Teenhome,3
Recency,100
MntWines,776


In [68]:
featuresWithLessThan20levels = df_value_counts.loc[df_value_counts["feature_levels"] <= 20]
categorical_list = featuresWithLessThan20levels.index
categorical_features_from_numerical = numerical.loc[:, categorical_list]
categorical_features_from_numerical.dtypes

Kidhome                  int64
Teenhome                 int64
NumDealsPurchases        int64
NumWebPurchases          int64
NumCatalogPurchases      int64
NumStorePurchases        int64
NumWebVisitsMonth        int64
AcceptedCmp3             int64
AcceptedCmp4             int64
AcceptedCmp5             int64
AcceptedCmp1             int64
AcceptedCmp2             int64
Complain                 int64
Z_CostContact            int64
Z_Revenue                int64
Customer_Tenure        float64
dtype: object

In [69]:
def convert_type(x, convert_to_type):
    x = x.astype(convert_to_type)
    return x
categorical_features_from_numerical = categorical_features_from_numerical.apply(lambda x : convert_type(x, "object"))
categorical_features_from_numerical.dtypes

Kidhome                object
Teenhome               object
NumDealsPurchases      object
NumWebPurchases        object
NumCatalogPurchases    object
NumStorePurchases      object
NumWebVisitsMonth      object
AcceptedCmp3           object
AcceptedCmp4           object
AcceptedCmp5           object
AcceptedCmp1           object
AcceptedCmp2           object
Complain               object
Z_CostContact          object
Z_Revenue              object
Customer_Tenure        object
dtype: object